Group Members: <br />
Abhiraj Khare -           2020A7PS0161H <br />
Nishith Kumar Gupta-          2020A4PS2336H <br />
Ananya Gautam - 2020AAPS2096H <br />

# NB and Logistic Regression Project 

Welcome to NB and Logistic Regression Project! Go ahead and follow the directions below.

**Build a classification model which will recognise the activities through the mobile sensor like walking, walking_upstairs  ,walking_downstairs ,sitting  ,standing,lying.The topic Human Activity recognition is in research and has its own advantages like anomaly detection, for healthy diet maintenance fitness bands are used , measuring stress level ,for monitoring employees ,for heartbeat pulse rate etc. In this project we are going to focus on  the dataset which was carried out through experiments with a group of 30 volunteers wearing a smartphone.The experiments have been video-recorded to label the data manually.The  main motto is to use the dataset and work with the libraries like scikit learn and machine learning related libraries and algorithms while training.**

# <font color='red'>Download and submit only HTML file</font>

# Import Libraries
**Import pandas,seaborn, and the usual libraries.**

In [1]:
import numpy as np
import pandas as pd 
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression

# EDA and Basic Data Preprocessing and Preparation [Marks:2]
**If trying auto EDA do it in a different notebook and only highlight important, interesting or unusual pattern that you noticed in the data**

<font color='red'>Give justification for your choices and techniques used for every step</font>

In [2]:
df = pd.read_csv("./ANANYA GAUTAM - human_activity.csv")

Looking at correlation between each features of our dataset and then removing the the columns which have their correlation higher than a certain threshold which is about 0.80 and also having many correlated columns is a redundancy which we do not need to use.
For any correlation above we will look to drop the column or think of doing the same and the threshold is 0.6

In [3]:
corr_mat = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > 0.80) or any(upper[column]< -0.80)]

## Null/Outlier Treatment

#### There are no null in the data set which have been seen from the auto eda report
#### There are no such outliers because the data can be skewed for different kind of activities
#### As we have concluded that there are no outliers and null so we dont need to do any treatment for the same

## Remove non-Important Features

This our actual data frame used for the model

In [4]:
new_df =  df.drop(df[to_drop],axis = 1)

The subject feature have no effect on model training hence it is removed

In [5]:
new_df = new_df.drop(new_df[['subject']],axis = 1)
new_df

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1",...,fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)",Activity
0,0.288585,-0.020294,-0.132905,-0.995279,0.929294,-0.058526,0.256892,0.264106,-0.095246,0.278851,...,-1.000000,0.586156,-1.000000,-0.074323,-0.298676,-0.112754,0.030400,-0.464761,-0.018446,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,0.611627,0.284595,0.115705,0.294310,-0.281211,0.085988,...,-0.948718,-0.336310,-1.000000,0.158075,-0.595051,0.053477,-0.007435,-0.732626,0.703511,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,0.273025,-0.164739,0.017150,0.342256,-0.332564,0.239281,...,-0.794872,-0.535352,-0.555556,0.414503,-0.390748,-0.118559,0.177899,0.100699,0.808529,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,0.061436,-0.264307,0.072545,0.323154,-0.170813,0.294938,...,-1.000000,-0.230091,-0.936508,0.404573,-0.117290,-0.036788,-0.012892,0.640011,-0.485366,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,0.313276,0.086904,0.257615,0.434728,-0.315375,0.439744,...,-0.897436,-0.510282,-0.936508,0.087753,-0.351471,0.123320,0.122542,0.693578,-0.615971,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,-0.561607,-0.056751,-0.094106,-0.119821,0.293112,-0.425386,...,-0.897436,-0.170845,-0.904762,-0.070157,-0.588433,-0.190437,0.829718,0.206972,-0.425619,WALKING_UPSTAIRS
7348,0.273853,-0.007749,-0.147468,-0.235309,-0.361660,-0.154722,0.032725,0.034260,0.239835,-0.364480,...,-0.846154,0.019626,-0.904762,0.165259,-0.390738,0.064907,0.875679,-0.879033,0.400219,WALKING_UPSTAIRS
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.376773,-0.017019,-0.004323,0.119962,0.080689,-0.420093,...,-0.846154,0.490169,-0.904762,0.195034,0.025145,0.052806,-0.266724,0.864404,0.701169,WALKING_UPSTAIRS
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.475165,-0.038354,-0.277801,0.101761,-0.108375,-0.438356,...,-0.846154,0.308105,-0.904762,0.013865,0.063907,-0.101360,0.700740,0.936674,-0.589479,WALKING_UPSTAIRS


## Split Predictor and Target Varaibles

In [6]:
new_df_shuffled=new_df.sample(frac=1).reset_index(drop=True)
new_df_shuffled
target = new_df_shuffled['Activity']
target

0       WALKING_UPSTAIRS
1       WALKING_UPSTAIRS
2               STANDING
3                SITTING
4                SITTING
              ...       
7347    WALKING_UPSTAIRS
7348              LAYING
7349            STANDING
7350             SITTING
7351             WALKING
Name: Activity, Length: 7352, dtype: object

Shuffling is required in order to avoid class exclusion in the training and testing dataset

In [7]:
new_df = new_df_shuffled.drop(new_df_shuffled[['Activity']],axis = 1)
new_df

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1",...,fBodyBodyGyroMag-min(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)"
0,0.194867,0.005240,-0.148642,-0.248229,-0.503322,0.253243,-0.411673,-0.195599,0.235627,-0.403223,...,-0.815741,-0.846154,-0.109966,-0.968254,-0.178301,-0.595599,0.566104,-0.627754,0.435882,0.265142
1,0.152418,-0.013601,-0.108788,-0.383088,-0.603644,0.260030,-0.364452,-0.058551,-0.055969,-0.761860,...,-0.690901,-0.846154,-0.152868,-0.904762,-0.049508,-0.379387,0.694499,-0.107402,0.934983,-0.386551
2,0.303895,-0.008799,-0.072297,-0.988837,0.050800,-0.109533,-0.215694,-0.026711,0.168744,-0.142150,...,-0.988742,-1.000000,0.137060,-1.000000,-0.126406,0.005000,0.012481,0.217165,-0.884383,0.889698
3,0.277415,-0.020079,-0.103354,-0.993947,0.165104,0.259351,0.142693,0.305290,-0.175596,0.124254,...,-0.995179,-0.897436,-0.490688,-1.000000,0.247958,-0.387659,-0.050972,-0.227317,-0.261269,-0.251444
4,0.276051,-0.013733,-0.095749,-0.998922,0.324619,-0.084472,0.126858,0.283821,-0.245798,0.165777,...,-0.997886,-1.000000,-0.150224,-0.714286,0.733301,-0.743342,0.022586,-0.101890,0.208852,0.570727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.359518,-0.016890,-0.077857,-0.279023,-0.324495,0.156242,-0.107176,0.168368,-0.023733,-0.179248,...,-0.887988,-0.948718,-0.240693,-0.904762,0.221643,-0.554102,-0.487089,0.826768,-0.712479,0.681961
7348,0.277909,-0.016479,-0.109866,-0.990494,0.033933,0.307533,0.246656,0.081708,0.429488,0.177910,...,-0.997632,-1.000000,-0.392010,-1.000000,-0.138512,-0.335424,-0.359332,-0.065855,0.288838,-0.196364
7349,0.280142,-0.013232,-0.107645,-0.997007,0.125903,0.304859,-0.185398,-0.210376,0.548652,0.124299,...,-0.968709,-1.000000,-0.237562,-0.936508,-0.065614,-0.520383,-0.010668,0.100817,-0.155080,-0.734500
7350,0.275465,-0.007687,-0.089049,-0.996788,0.325564,0.051237,-0.069927,-0.101236,0.101055,0.132462,...,-0.991195,-0.948718,-0.483195,-0.968254,0.165080,-0.187510,0.063827,0.018399,0.167838,0.292683


New DataFrame (new_df) is the feature vector space and target is the alleged class output

## Split Training and Testing Data Set

#### With less training data the parameters estimates high variance. On the other hand, less testing data leads to high variance on performance measure.
#### And thus we used 78:22 split of the dataset because of its flexibility in giving good size of testing data and training data which minimizes the variance on performance measure. 
#### Also shuffling the data helps with non-clustering of the target variables
#### We came to this solution by running experimentation on our models with different train size from 0.60 to 0.95.

In [8]:
train_x,test_x,train_y,test_y = train_test_split(new_df,target,train_size = 0.78,random_state = 42)

## Standardize the Variables

In [9]:
train_x = StandardScaler().fit_transform(train_x)
test_x = StandardScaler().fit_transform(test_x)

Standardized data:
Standarized_X = (X - mean(X))/ std(X)

# Train Model using different variations of NB [Marks:2+1]
**Breifly describe & Justify why one of the variant performed better compared to others**

In [10]:
# Gaussian Naive Bayes
gnb = GaussianNB()
begin = time.time()
y_pred_gnb = gnb.fit(train_x,train_y).predict(test_x)
end = time.time()
Gauss_time =  begin - end
print("Time = " + str(end - begin) + " sec")
print("Accuracy = " + str(accuracy_score(test_y, y_pred_gnb)))

Time = 0.06839179992675781 sec
Accuracy = 0.7608158220024722


In [11]:
# Bernoulli Naive Bayes
cmb = BernoulliNB()
begin = time.time()
y_pred_cmb = cmb.fit(train_x,train_y).predict(test_x)
end = time.time()
Bernoulli_time =  begin - end 
print("Time = " + str(end - begin) + " sec")
print("Accuracy = " + str(accuracy_score(test_y, y_pred_cmb)))

Time = 0.06400799751281738 sec
Accuracy = 0.8374536464771323


#### Gaussian Distribution performed better compared to the Bernoulli algorithm is because Gaussian uses the Gaussian Distribution for building the model and our data
#### after standardising ( In the earlier code ) works closer to Gaussian distribution and hence we obtain better results.

# Compare time and accuracy of different NB algos [Marks:0.5]

#### Time for Gaussian was lesser than that for Bernoulli, as well as accuracy for Gaussian was better.
#### As from the output we can see the time and the accuracy for the Gaussian naive bayes algorithm is better by a good amount of difference
#### We couldnt use other naive bayes algorithm because categorical, complement and multinomial algorithm uses either binary inputs or cannot accept negative values for the same

# Train Model with Logistic Regression with different solver and penalty [Marks:2+1]
**Justify why one of the solver or penalty performed better compared to others**

#### there are three types of penalities supported by the LogistticRegressor , which are L1, L2 ans ElasticNet. 
#### L2 is default choice which is automatically implemented by sklearn if nothing is mentioned as the penalty parameter.
#### lbfgs, newton-cg, liblinear, saga are the three types of solvers used.
#### lbfgs and newton-cg use L2 regularization or none.
#### Where as, liblinear can use both l1 and l2 penalty.
#### Saga solver uses elasticNet as its penalty. 

In [12]:
begin = time.time()
clf = LogisticRegression(multi_class = "multinomial", solver = "lbfgs", random_state=0, penalty="l2").fit(train_x, train_y)
end = time.time()
print("Time = " + str(end - begin) + " sec")
y_pred_logistic_ls = clf.predict(test_x)
print("Accuracy = " + str(accuracy_score(test_y, y_pred_logistic_ls)))

Time = 0.5105669498443604 sec
Accuracy = 0.9752781211372065


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
from sklearn.linear_model import LogisticRegression
begin = time.time()
clf = LogisticRegression(solver = "liblinear", random_state=0, penalty="l2").fit(train_x, train_y)
end = time.time()
print("Time = " + str(end - begin) + " sec")
y_pred_logistic_lb = clf.predict(test_x)
print("Accuracy = " + str(accuracy_score(test_y, y_pred_logistic_lb)))

Time = 1.4173469543457031 sec
Accuracy = 0.9796044499381953


In [14]:
from sklearn.linear_model import LogisticRegression
begin = time.time()
clf = LogisticRegression(solver = "liblinear", random_state=0, penalty="l1").fit(train_x, train_y)
end = time.time()
print("Time = " + str(end - begin) + " sec")
y_pred_logistic_lb = clf.predict(test_x)
print("Accuracy = " + str(accuracy_score(test_y, y_pred_logistic_lb)))

Time = 2.36022686958313 sec
Accuracy = 0.9783683559950557


In [15]:
from sklearn.linear_model import LogisticRegression
begin = time.time()
clf = LogisticRegression(multi_class = "multinomial", solver = "newton-cg", random_state=0).fit(train_x, train_y)
end = time.time()
print("Time = " + str(end - begin) + " sec")
y_pred_logistic_nc = clf.predict(test_x)
print("Accuracy = " + str(accuracy_score(test_y, y_pred_logistic_nc)))

Time = 1.3653218746185303 sec
Accuracy = 0.9746600741656366


In [16]:
from sklearn.linear_model import LogisticRegression
begin = time.time()
clf = LogisticRegression(multi_class = "multinomial", solver = "saga", random_state=0, penalty="elasticnet", l1_ratio = 0.3).fit(train_x, train_y)
end = time.time()
print("Time = " + str(end - begin) + " sec")
y_pred_logistic_sg = clf.predict(test_x)
print("Accuracy = " + str(accuracy_score(test_y, y_pred_logistic_sg)))

Time = 2.4442780017852783 sec
Accuracy = 0.9734239802224969


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


# Compare time and accuracy of different Logistic Regression algos [Marks:0.5]

#### We compare the running time for building all the different classifiers using the Logistic Classification Algos, and find that The Logistic Regression Model 
#### for lbfgs solver and L2 penalty gives the minimum time and the best accuracy. 
#### Though Newton-cg solver gave slightly better accuracy as compared to lbfgs, it was computationally expensive thus taking longer time to fit the model. 
#### Also, liblinear with L1 regularization gives a better accuracy but takes very long time to fit the model. 
#### Hence we go with solver = lbfgs and L2 penalty. 

Minimum time =  lbfgs (0.510s)
Maximum Accuracy = liblinear_l2 (97.9604%)

# Plot confusion Matrix for the final model selected [Marks:1]

In [17]:
print("Confusion Matrix : \n")
print(confusion_matrix(test_y, y_pred_logistic_ls))

Confusion Matrix : 

[[318   0   0   0   0   0]
 [  1 267  20   0   0   0]
 [  0  13 309   0   0   0]
 [  0   0   0 250   0   0]
 [  0   0   0   1 207   2]
 [  0   0   0   1   2 227]]


Final model selected is lbfgs because it gives a comaparable accuracy with much better computation expense

In [18]:
print("Precision = " + str(precision_score(test_y,y_pred_logistic_ls, average = 'micro')))
print("Recall = " + str(recall_score(test_y,y_pred_logistic_ls, average = 'micro')))
print("F1 Score = " + str(f1_score(test_y,y_pred_logistic_ls, average = 'micro')))

Precision = 0.9752781211372065
Recall = 0.9752781211372065
F1 Score = 0.9752781211372065


In [19]:
print("Percision = " + str(precision_score(test_y,y_pred_logistic_ls, average = 'macro')))
print("Recall = " + str(recall_score(test_y,y_pred_logistic_ls, average = 'macro')))
print("F1 Score = " + str(f1_score(test_y,y_pred_logistic_ls, average = 'macro')))

Percision = 0.9772344747326444
Recall = 0.9765635783298827
F1 Score = 0.9768448763268913


In [20]:
print("Precision = " + str(precision_score(test_y,y_pred_logistic_ls, average = 'weighted')))
print("Recall = " + str(recall_score(test_y,y_pred_logistic_ls, average = 'weighted')))
print("F1 Score = " + str(f1_score(test_y,y_pred_logistic_ls, average = 'weighted')))

Precision = 0.9753119951502668
Recall = 0.9752781211372065
F1 Score = 0.9752356851162027


#### Micro score calculates the F1_score by considering total true positives, false negatives and false positive. 
#### Macro computes the F1_score for each label and then returns the average without considering the prportion for each label. 
#### Weighted computes F1_score for each label then computes the average considering the proportion for each label. 